In [1]:
%load_ext autoreload
%autoreload 2

# Docugami
This notebook covers how to load documents from `Docugami`. See [here](../../../../ecosystem/docugami.md) for more details, and the advantages of using this system over alternative data loaders.

## Prerequisites
1. Follow the Quick Start section in [this document](../../../../ecosystem/docugami.md)
2. Grab an access token for your workspace, and make sure it is set as the DOCUGAMI_API_KEY environment variable
3. Grab some docset and document IDs for your processed documents, as described here: https://help.docugami.com/home/docugami-api

In [ ]:
# you need the lxml package to use the DocugamiLoader
!pip install --upgrade lxml

In [2]:
import os
from langchain.document_loaders import DocugamiLoader

## Load Documents

If the DOCUGAMI_API_KEY environment variable is set, there is need to pass it in to the loader explicitly otherwise you can pass it in as the `access_token` parameter.

In [19]:
DOCUGAMI_API_KEY=os.environ.get('DOCUGAMI_API_KEY')
loader = DocugamiLoader(docset_id="rodbqb1eb0yr") # load all docs in the given docset ID
docs = loader.load()
docs

[Document(page_content='MUTUAL NON-DISCLOSURE AGREEMENT', metadata={'xpath': '/docset:MUTUALNON-DISCLOSUREAGREEMENT-section/docset:MutualNon-disclosureAgreement', 'documentId': 'pknha3yod0kq', 'structure': 'h1', 'tag': 'MutualNon-disclosureAgreement', 'projects': [{'projectType': 'ExcelExport', 'projectTitle': 'Test Report', 'entries': [{'heading': 'Effective Date', 'xpath': '/docset:MUTUALNON-DISCLOSUREAGREEMENT-section/docset:MUTUALNON-DISCLOSUREAGREEMENT/docset:ThisMutualNon-disclosureAgreement/docset:ThisMutualNon-disclosureAgreement/docset:EffectiveDate', 'value': '2/4/2018'}, {'heading': 'Signatory Name', 'xpath': '/docset:MUTUALNON-DISCLOSUREAGREEMENT-section/docset:MUTUALNON-DISCLOSUREAGREEMENT/docset:ThisMutualNon-disclosureAgreement/docset:ThisMutualNon-disclosureAgreement/docset:SignatoryName', 'value': 'Leonarda Hosler'}]}]}),
 Document(page_content="This  Mutual Non-Disclosure Agreement  (this “ Agreement ”) is entered into and made effective as of  2/4/2018  between  Docu